In [1]:
%run "../data_tools.ipynb"
%run "../create_metrics.ipynb"

      T_ID   GDAY_DS  HEADER_NO   P_ID  START_CK  BAT_ORDER_NO   PA   AB  RBI  \
0       HH  20160401          0  60404         0             3    1    1    0   
1       HH  20160401          0  62700         1             9    2    2    0   
2       HH  20160401          0  64086         1             7    6    4    0   
3       HH  20160401          0  66740         1             5    6    6    0   
4       HH  20160401          0  71347         1             2    6    6    1   
...    ...       ...        ...    ...       ...           ...  ...  ...  ...   
18679   WO  20161009          0  74215        91           374  402  341   80   
18680   WO  20161009          0  78168       139           177  646  560   63   
18681   WO  20161009          0  79130        15           251   80   66    7   
18682   WO  20161009          0  79300        13           429  106   91    9   
18683   WO  20161009          0  79365       122           965  454  411   70   

       RUN  ...  BB  IB  HP

In [ ]:
crawled_player_batter = pd.read_csv("../Crawler/crawled_player_batter.csv")
crawled_player_pitcher = pd.read_csv("../Crawler/crawled_player_pitcher.csv")
crawled_team_batter = pd.read_csv("../Crawler/crawled_team_batter.csv")
crawled_team_pitcher = pd.read_csv("../Crawler/crawled_team_pitcher.csv")
crawled_csum_player_pitcher = pd.read_csv("../Crawler/crawled_csum_player_pitcher.csv")

crawled_player_batter = crawled_player_batter.drop(["TB"], axis = "columns")
crawled_team_batter = crawled_team_batter.drop(["TB", "START_CK"], axis = "columns")
crawled_player_pitcher = crawled_player_pitcher.drop(["WHIP"], axis = "columns")
crawled_team_pitcher = crawled_team_pitcher.drop(["WHIP"], axis = "columns")
crawled_csum_player_pitcher = crawled_csum_player_pitcher.drop(["WHIP", "MONEY", "AGE_VA", "NAME"], axis = "columns")

def pitcher_data(T_ID) :
    total_data = []
    for i in list(range(2016, 2021)) :
        rawData = load_data(i)
        Scaling_Money(rawData["player"])
        rawData["player_batter"] = pd.merge(rawData["player_batter"], rawData["player"][["P_ID", "T_ID", "AGE_VA", "MONEY", "NAME"]])
        rawData["player_pitcher"] = pd.merge(rawData["player_pitcher"], rawData["player"][["P_ID", "T_ID", "AGE_VA", "MONEY", "NAME"]])

        rawData["player_batter"] = rawData["player_batter"][crawled_player_batter.columns]
        rawData["player_pitcher"] = rawData["player_pitcher"][crawled_player_pitcher.columns]
        rawData["team_batter"] = rawData["team_batter"][crawled_team_batter.columns]
        rawData["team_pitcher"] = rawData["team_pitcher"][crawled_team_pitcher.columns]
        rawData["csum_player_pitcher"] = rawData["csum_player_pitcher"][crawled_csum_player_pitcher.columns]

        if i == 2020 :
            rawData["player_batter"] = pd.concat([rawData["player_batter"], crawled_player_batter]).reset_index(drop = True)
            rawData["player_pitcher"] = pd.concat([rawData["player_pitcher"], crawled_player_pitcher]).reset_index(drop = True)
            rawData["team_batter"] = pd.concat([rawData["team_batter"], crawled_team_batter]).reset_index(drop = True)
            rawData["team_pitcher"] = pd.concat([rawData["team_pitcher"], crawled_team_pitcher]).reset_index(drop = True)
            rawData["csum_player_pitcher"] = pd.concat([rawData["csum_player_pitcher"], crawled_csum_player_pitcher]).reset_index(drop = True)

        make_batter_metrics(rawData, "player_batter")
        make_batter_metrics(rawData, "team_batter")
        make_pitcher_metrics(rawData, "player_pitcher")
        make_pitcher_metrics(rawData, "team_pitcher")
        total_data.append(rawData)

    tableList = []
    for key in total_data[0].keys() :
        tableList.append(key)

    final_data = dict()
    for key in tableList :
        final_data[key] = pd.DataFrame()

    for year in range(len(total_data)) :
        for key in tableList :
            if final_data[key].empty :
                final_data[key] = total_data[year][key]
            else :
                final_data[key] = pd.concat([final_data[key], total_data[year][key]], axis = 0)

    for key in final_data.keys() :
        final_data[key].reset_index(drop = True)
    #########################################
    final_data["player_pitcher"].rename(columns = {"MONEY" : "MONEY_PIT", "AGE_VA" : "AGE_VA_PIT"}, inplace = True)
    final_data["player_pitcher"] = final_data["player_pitcher"].reset_index(drop = True)

    metrics_pitcher = ["BB/9", "LOB%", "FIP", "DER", "ERC", "ERA", "WHIP", "RA/9", "MONEY_PIT", "AGE_VA_PIT"]
    final_data["player_pitcher"].sort_values(by = ["T_ID", "GDAY_DS", "HEADER_NO"])
    final_data["player_pitcher"] = final_data["player_pitcher"].reset_index(drop = True)

    result = final_data["player_pitcher"].groupby(["GDAY_DS", "T_ID", "HEADER_NO"])["INN2"].sum()
    result = result.to_frame().dropna().reset_index()
    result.rename(columns = {"INN2" : "INN2_SUM"}, inplace = True)
    final_data["team_pitcher"] = pd.get_dummies(final_data["team_pitcher"], columns = ["WLS"], drop_first = True)

    final_data["player_pitcher"] = pd.merge(final_data["player_pitcher"], result)
    final_data["player_pitcher"][metrics_pitcher] = final_data["player_pitcher"][metrics_pitcher].mul((final_data["player_pitcher"]["INN2"] / final_data["player_pitcher"]["INN2_SUM"]), axis = 0)
    final_data["player_pitcher"] = final_data["player_pitcher"].drop("INN2_SUM", axis = "columns")
    final_data["player_pitcher"] = final_data["player_pitcher"][final_data["player_pitcher"]["INN2"] != 0]
    final_data["player_pitcher"] = final_data["player_pitcher"][~np.isinf(final_data["player_pitcher"]["LOB%"])]

    ##########################################
    pitcher = final_data["player_pitcher"].groupby(["GDAY_DS", "T_ID", "HEADER_NO"]).sum()
    pitcher.rename(columns = {"ERA" : "ERA_WEIGHTED"}, inplace = True)
    pitcher = pitcher.drop(["START_CK", "PPF"], axis = "columns")
    pitcher = pitcher.dropna(axis = 0).reset_index(inplace = False)

    pitcher = pd.merge(pitcher, final_data["team_pitcher"][["GDAY_DS", "T_ID", "HEADER_NO", "PPF", "ERA", "WLS_W"]])
    pitcher = pitcher.sort_values(by = ["T_ID", "GDAY_DS"]).reset_index(inplace = False, drop = True)
    ###########################################
    target = pitcher[pitcher["T_ID"] == "HH"].dropna().ERA[20:].reset_index(drop = True)

    feature = pitcher[pitcher["T_ID"] == "HH"].dropna().drop(["GDAY_DS", "T_ID", "HEADER_NO", "H3", "IB", "BF", "H2", "HP", "WLS_W",
                                                             "BB", "KK", "INN2", "HR"], axis = "columns").reset_index(drop = True)

    pca = PCA(n_components = 1)
    res = pca.fit_transform(feature[["ERA", "R", "ER", "RA/9", "ERA_WEIGHTED", "WHIP", "ERC"]])
    feature["ER_PCA"] = res
    feature = feature.drop(["ERA", "R", "ER", "RA/9", "ERA_WEIGHTED", "WHIP", "ERC"], axis = "columns")

    pca = PCA(n_components = 1)
    res = pca.fit_transform(feature[["PA", "AB", "HIT"]])
    feature["BT_PCA"] = res
    feature = feature.drop(["PA", "AB", "HIT"], axis = "columns")

    feature = feature.loc[0:len(target)-1]
    return feature, target
